In [23]:

def above_power_2(x):
    i = 1
    while i<x:
        i *= 2
    return i

def total_sum_seq(a):
    s = 0
    for i in range(len(a)):
        s += a[i]
    return s

def total_sum_folding(a):
    l_a = above_power_2(len(a))
    d = len(a)//2
    while d>0:
        
        for thread in range(d):
            a[thread] += a[thread+d]
        d = d//2
            
    return
  
# test procedures

def fill_array(a):
    for i in range(len(a)):
        a[i] = i
    return a

SIZE = 1024
a = [0]*SIZE
fill_array(a)
print(total_sum_seq(a))
total_sum_folding(a)
print(a[0])


523776
523776
